In [4]:
!brew install libpq


zsh:1: command not found: brew


In [1]:
import pinecone
import pandas as pd
import timeit
from transformers import AutoTokenizer, AutoModel
import torch

class PineconeWrapper:
    def __init__(self, api_key, environment, index_name, dimension):
        pinecone.init(api_key=api_key, environment=environment)
        self.index = pinecone.Index(index_name)
        self.report = {}
        if index_name not in pinecone.list_indexes():
            pinecone.create_index(index_name, dimension=dimension)
        self.model_name = "sentence-transformers/all-MiniLM-L6-v2"
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        self.model = AutoModel.from_pretrained(self.model_name)

    def text_to_vector(self, text):
        inputs = self.tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
        with torch.no_grad():
            outputs = self.model(**inputs)
        return outputs.pooler_output[0].numpy()

    def add_documents(self, data):
        vectors = [
            {'id': str(i), 'values': self.text_to_vector(row['TEXTO']).tolist(), 'metadata': {'titulo': row['TITULO'], 'autor': row['AUTOR']}}
            for i, row in data.iterrows()
        ]
        start_time = timeit.default_timer()
        self.index.upsert(vectors=vectors)
        end_time = timeit.default_timer()
        self.report["building time"] = end_time - start_time

    def query_documents(self, query_text, top_k=1):
        query_vector = self.text_to_vector(query_text)
        start_time = timeit.default_timer()
        self.index.query(query_vector.tolist(), top_k=top_k)
        end_time = timeit.default_timer()
        self.report["query time"] = end_time - start_time

    def calculate_query_latency(self, query_text, num_queries, top_k=1):
        total_time = 0
        for _ in range(num_queries):
            start_time = timeit.default_timer()
            self.index.query(self.text_to_vector(query_text).tolist(), top_k=top_k)
            total_time += timeit.default_timer() - start_time
        self.report["query latency"] = total_time / num_queries
        return self.report["query latency"]
    
    def calculate_qps(self, query_text, num_queries, top_k=1):
        start_time = timeit.default_timer()
        for _ in range(num_queries):
            self.index.query(self.text_to_vector(query_text).tolist(), top_k=top_k)
        end_time = timeit.default_timer()
        qps = num_queries / (end_time - start_time)
        self.report["queries per second"] = qps
        return qps

    def get_report(self):
        for key in self.report:
            print(f"{key}: {self.report[key]}")


pinecone_wrapper = PineconeWrapper(api_key='396914a9-a013-4cd9-b65a-8feda57ae7c1', 
                                   index_name='pinecone-index',
                                   environment= "gcp-starter", 
                                   dimension=384)

data = pd.read_csv('../data/textos.csv')
pinecone_wrapper.add_documents(data)
pinecone_wrapper.calculate_query_latency('¿Quién es Quijote de la Mancha?', 5)
pinecone_wrapper.query_documents("This is a query document")
pinecone_wrapper.calculate_qps('¿Quién es Quijote de la Mancha?', 100)
pinecone_wrapper.get_report()


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [9]:
pinecone_wrapper = PineconeWrapper(api_key='396914a9-a013-4cd9-b65a-8feda57ae7c1', 
                                   index_name='pinecone-index',
                                   environment= "gcp-starter", 
                                   dimension=384)

data = pd.read_csv('../data/textos.csv')
pinecone_wrapper.add_documents(data)
pinecone_wrapper.calculate_query_latency('¿Quién es Quijote de la Mancha?', 5)
pinecone_wrapper.query_documents("This is a query document")
pinecone_wrapper.calculate_qps('¿Quién es Quijote de la Mancha?', 100)
pinecone_wrapper.get_report()


building time: 1.6703985409985762
query latency: 0.3170276749995537
query time: 0.1981152500011376
queries per second: 4.231311206809164
